In [2]:
%load_ext autoreload
%autoreload 2

from computage.deage.kdm import KlemeraDoubalEstimator
from computage.deage.pls import PLS1
from computage.utils.data_utils import test_dataset
from computage.utils.metric_utils import print_metrics
import pickle

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#other train datasets
# 
train_list = ['GSE40279', 'GSE132203', 'GSE51032']


train_data = pd.DataFrame()
train_meta = pd.DataFrame()

for gse in train_list:
    data, meta = pd.read_pickle('/tank/projects/computage/train/GSE40279.pkl.gz', compression='gzip').values()

In [4]:
# from biolearn.data_library import DataLibrary
# lib = DataLibrary()
# data_source = lib.get('GSE51032')
# data = data_source.load()

In [23]:
# raw_dataset = {}
# raw_dataset['data'] = data.dnam.T.astype('float32')
# raw_dataset['meta'] = data.metadata
# pd.to_pickle(raw_dataset, f'/tank/projects/computage/train/GSE51032.pkl.gz', compression='gzip')

In [3]:
data, meta = pd.read_pickle('/tank/projects/computage/train/GSE40279.pkl.gz', compression='gzip').values()

In [4]:
X = data.copy()
y = meta['age'].copy()

In [6]:
from biolearn.model_gallery import ModelGallery
gallery = ModelGallery()
list(gallery.model_definitions.keys())
k1 = 'HRSInCHPhenoAge'
k2 = 'PhenoAge'

indices = gallery.get(k1).clock.coefficients.index

In [12]:
stats.to_csv('/tank/projects/computage/train/train_stats/GSE40279_corrstats.csv')

In [25]:
stats['fi'] = np.abs(stats['rvalue'])**2 * stats['r_res']**2
#stats['fi'] = np.abs(stats['rvalue'])**2 / stats['rse']**2 * stats['r_res']**2
# stats['fi'] = stats['r_res']**2 / stats['rse']**2

stats.sort_values('fi', ascending=False).head(7)

,slope,intercept,rvalue,p-value,stderr,rse,r2,s_res,i_res,r_res,p_res,fi
id,,,,,,,,,,,,
cg21585707,-0.002072,1.012886,-0.461535,6.434417e-36,0.000156,0.058722,0.213014,0.001176,-0.032820,0.428474,1.130394e-30,0.039107
cg02650266,0.002279,-0.004639,0.628439,2.207339e-73,0.000110,0.041610,0.394935,0.000580,-0.006996,0.298569,5.656864e-15,0.035206
cg05898618,-0.003374,1.008246,-0.500123,8.215360e-43,0.000228,0.086163,0.250123,0.001240,-0.011056,0.349607,2.704387e-20,0.030571
cg07955995,0.001615,-0.028346,0.553357,6.798293e-54,0.000095,0.035849,0.306203,0.000546,-0.010653,0.306399,1.002678e-15,0.028747
cg02228185,-0.003830,0.961347,-0.567026,4.486228e-57,0.000218,0.082062,0.321519,0.001067,-0.005707,0.297279,7.484033e-15,0.028414
cg22285878,0.001013,-0.008345,0.592456,2.104541e-63,0.000054,0.020314,0.351004,0.000260,-0.001540,0.282749,1.592633e-13,0.028062
cg14361627,0.002156,0.095335,0.613768,3.700989e-69,0.000108,0.040899,0.376711,0.000500,-0.001864,0.267852,3.056270e-12,0.027027


In [16]:
# corrcoefs = X.corrwith(y)

In [26]:
# sorted_sites = corrcoefs.abs().sort_values(ascending=False)
sorted_sites = stats.sort_values('fi', ascending=False)
top_sites = sorted_sites.iloc[:20000].index #sorted_sites[sorted_sites > 0.25].index     #arbitrary threshold !!!

In [7]:
Xtop = X[indices]

In [42]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(Xtop, y, test_size=0.15)

In [9]:
n_components = np.arange(1, 14)

for n in n_components:
    pls = PLS1(n_components=n)
    pls.fit(Xtop, y)
    pickle.dump(pls, open(f'/tank/projects/computage/checkpoints/pls1_{n}_pheno.pkl', 'wb'))

In [10]:
#PLS with KDM head

n_components = np.arange(1, 14)

for n in n_components:
    pls = PLS1(n_components=n, head_estimator='kdm')
    pls.fit(Xtop, y)
    pickle.dump(pls, open(f'/tank/projects/computage/checkpoints/pls1_{n}_kdm_pheno.pkl', 'wb'))

In [24]:
selection_method = 'forward'
weighings = ['rse', 'r2', 'r2|rse', None]
lasso=True

for w in weighings:
    kdm = KlemeraDoubalEstimator(cv=5, cv_val_size=0.2,
                                feature_selection_method=selection_method,
                                feature_selection_criterion='mse',
                                feature_pval_threshold=0.01,
                                lasso_preselection=lasso,
                                weighing=w,
                                max_features=10000,
                                n_jobs=8,
                                verbose=1)

    kdm.fit(Xtop, y)
    pickle.dump(kdm, open(f'/tank/projects/computage/checkpoints/kdm_{w}_{selection_method}_{str(lasso)}_20k.pkl', 'wb'))

Training estimators on full data.
20263 features remained after applying p-value threshold.
Run Lasso.


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
................................................................................................................................................................................................................./home/dkriukov/.conda/envs/computage/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.200e+01, tolerance: 9.259e+00
  model = cd_fast.enet_coordinate_descent(
............................[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:  1.4min remaining:  2.1min
......./home/dkriukov/.conda/envs/computage/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check

433 features remained after Lasso pre-selection.
Training estimators on different cv-folds.


100%|██████████| 433/433 [00:17<00:00, 24.08it/s]


Remove redundant features.
6 features remained after selection.
Bvar=36.103
Finished!
Training estimators on full data.
20263 features remained after applying p-value threshold.
Run Lasso.


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
......................................................................................................................................................................................................................./home/dkriukov/.conda/envs/computage/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.200e+01, tolerance: 9.259e+00
  model = cd_fast.enet_coordinate_descent(
..........................[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:  1.4min remaining:  2.1min
..../home/dkriukov/.conda/envs/computage/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, chec

433 features remained after Lasso pre-selection.
Training estimators on different cv-folds.


KeyboardInterrupt: 

In [17]:
pickle.dump(kdm, open(f'/tank/projects/computage/checkpoints/kdm_rse_all_20k.pkl', 'wb'))

In [15]:
kdm

,slope,intercept,rvalue,p-value,stderr,rse,r2,r_abs,fi,adj.p-value,weight
id,,,,,,,,,,,
cg16867657,0.004641,0.372663,0.858536,6.916502e-192,0.000108,0.040879,0.737084,0.858536,513.746823,1.401491e-187,0.040879
cg06639320,0.002811,0.294148,0.747103,4.178307e-118,0.000098,0.036882,0.558162,0.747103,549.222798,4.233252e-114,0.036882
cg24724428,0.003983,0.004570,0.744579,6.716295e-117,0.000140,0.052661,0.554398,0.744579,268.493148,4.536410e-113,0.052661
cg22454769,0.004058,0.334766,0.743994,1.272722e-116,0.000143,0.053747,0.553528,0.743994,257.552502,6.447291e-113,0.053747
cg10501210,-0.007904,1.034369,-0.726036,1.877941e-108,0.000293,0.110406,0.527128,0.726036,59.562840,7.610546e-105,0.110406
...,...,...,...,...,...,...,...,...,...,...,...
cg17338544,-0.000804,0.582865,-0.250025,8.304314e-11,0.000122,0.045908,0.062512,0.250025,118.630900,8.305954e-11,0.045908
cg13308279,-0.001133,0.450821,-0.250016,8.317061e-11,0.000172,0.064687,0.062508,0.250016,59.748853,8.318293e-11,0.064687
cg06518628,0.000378,0.085125,0.250013,8.321555e-11,0.000057,0.021581,0.062507,0.250013,536.819577,8.322376e-11,0.021581


# loading

In [75]:
pickle.dump(pls, open('/tank/projects/computage/checkpoints/pls1_20000.pkl', 'wb'))

In [1]:
import pickle

pls = pickle.load(open('/tank/projects/computage/checkpoints/pls1_20000.pkl', 'rb'))

In [2]:
pls

PLS1(n_components=11)

In [2]:
import pandas as pd
data, meta = pd.read_pickle('/tank/projects/computage/train/GSE40279.pkl.gz', compression='gzip').values()

In [12]:
data

id,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,ch.9.945770F,ch.9.96055087R,ch.9.97139671F,ch.9.98463211R,ch.9.98936572R,ch.9.98937537R,ch.9.98957343R,ch.9.98959675F,ch.9.98989607R,ch.9.991104F
GSM989827,0.464197,0.941091,0.911182,0.132014,0.717861,0.686521,0.805003,0.228244,0.338484,0.016508,...,0.022659,0.109918,0.061222,0.034284,0.133693,0.042808,0.052589,0.035624,0.028066,0.043850
GSM989828,0.454883,0.939033,0.596455,0.206917,0.723935,0.619084,0.814672,0.310879,0.418998,0.005747,...,0.005095,0.076996,0.052640,0.027978,0.125270,0.036811,0.053343,0.075618,0.017428,0.032950
GSM989829,0.485764,0.918802,0.870333,0.162861,0.719196,0.635678,0.824336,0.263215,0.424736,0.012197,...,0.021444,0.070694,0.058888,0.032643,0.139105,0.042844,0.045973,0.126421,0.021752,0.022375
GSM989830,0.480785,0.929908,0.889689,0.197780,0.704061,0.610864,0.811152,0.316761,0.398772,0.019945,...,0.028587,0.094749,0.056279,0.036997,0.140601,0.042258,0.048733,0.084051,0.027504,0.053007
GSM989831,0.501220,0.934548,0.890450,0.148437,0.754913,0.651262,0.808628,0.338289,0.366965,0.000000,...,0.018626,0.110543,0.057568,0.036746,0.129993,0.039613,0.039254,0.165874,0.020889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM990623,0.527496,0.958173,0.922034,0.223068,0.778959,0.709248,0.825768,0.354296,0.396241,0.001778,...,0.016319,0.079741,0.072076,0.052052,0.159212,0.049571,0.033060,0.093414,0.013654,0.014088
GSM990624,0.588331,0.982450,0.855145,0.162180,0.796869,0.535832,0.806714,0.295598,0.301319,0.006685,...,0.023837,0.117762,0.058466,0.037199,0.133899,0.055904,0.062442,0.101100,0.014193,0.000000
GSM990625,0.362994,0.954392,0.927184,0.196430,0.713020,0.664184,0.804045,0.395724,0.445179,0.003822,...,0.012054,0.079249,0.053499,0.050002,0.149589,0.048526,0.041087,0.049857,0.016840,0.041415
GSM990626,0.499145,0.931691,0.900938,0.167477,0.730215,0.665792,0.831365,0.338117,0.383953,0.000000,...,0.009878,0.090751,0.060335,0.040898,0.168269,0.056429,0.047835,0.028896,0.025346,0.052959


In [3]:
pls.predict(data[pls.pls.feature_names_in_])

array([ 66.42367723,  86.84638989,  66.09777852,  65.19766558,
        63.07946109,  79.57633079,  72.89482126,  73.80297895,
        73.52725308,  83.45285966,  81.24766192,  47.34764094,
        77.73492165,  54.23932683,  64.81466742,  70.91454628,
        67.92215057,  79.9651596 ,  89.96608444,  61.09454018,
        79.08460445,  68.22490182,  93.57434465,  83.33718701,
        72.67404202,  74.72274771,  73.25616846,  55.31872063,
        72.36517662,  76.5079233 ,  79.65530652,  73.30333896,
        81.20696417,  73.92909182,  79.9276413 ,  55.85773384,
       100.68049539,  80.01059737,  83.12165139,  86.02318938,
        83.96654195,  76.37917872,  77.30844778,  73.50498884,
        86.77273412,  83.58304909,  87.73991971,  64.28288229,
        78.24121367,  55.79925848,  75.09902065,  66.66558865,
        72.44815166,  56.59411679,  75.94694576,  96.93307961,
        56.76141275,  75.46924962,  46.21147077,  57.7864869 ,
        65.02392686,  84.15695736,  63.02168176,  89.03